In [1]:
!pip install PySastrawi
from bs4 import BeautifulSoup
import requests
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import tensorflow as tf
import numpy as np
import sklearn
from keras_preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 210 kB 5.2 MB/s 


# Scrapping Data

In [ ]:
#scrapping news
import bs4
URL = "https://turnbackhoax.id/category/fitnah-hasut-hoax/page/"
upperframe = []

for page in range(1,421):
  data = []
  req = requests.get(URL + str(page) + "/")
  soup = BeautifulSoup(req.text, "html.parser")

  coverpage_news = soup.find_all("h3", attrs = {"class" : "entry-title mh-loop-title"})
  for j in coverpage_news:
    judul = j.find("a").get_text().strip()
    link = j.find("a")["href"].strip()
    data.append([judul, link])
  upperframe.extend(data)  

In [ ]:
#create dataframe for scrapping news
data = pd.DataFrame(upperframe, columns=["judul","link"])
data.head(10)

## Data Preprocessing

In [ ]:
real_train = pd.read_excel("real_train.xlsx")
real_test = pd.read_excel("real_testing.xlsx")
fake_train = pd.read_excel("fake_train.xlsx")
fake_test = pd.read_excel("fake_testing.xlsx")
test = pd.read_csv("Data_latih.csv")

Data Training

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
real_train["label"] = 0
real_test["label"] = 0
fake_train["label"] = 1
fake_test["label"] = 1

In [ ]:
train = pd.concat([real_train, fake_train], ignore_index=True)
validation = pd.concat([real_test, fake_test], ignore_index=True)

In [ ]:
#Preprocessing data training
#remove punctuation
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
def remove_punctuation(text):
    no_punct = "".join([i for i in text if i not in punc])
    return no_punct

train['judul'] = train['judul'].apply(lambda x: remove_punctuation(x))

#lowering text
train['judul'] = train['judul'].apply(lambda x: x.lower())

#remove number
def remove_number(text):
    no_number = re.sub(r'\d', '', text)
    return no_number

train['judul'] = train['judul'].apply(lambda x: remove_number(x))

#remove emoji
train["judul"] = train["judul"].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))

#remove whitespace
# def remove_whitespace(text):
#     if len(text.split()) > 1:
#         result = ' '.join(text.split())
#         return result

# df['judul'] = df['judul'].apply(remove_whitespace)
# df.head(10)

#define stemming
factory = StemmerFactory()
stemmer = factory.create_stemmer()

#define stopwords
stopword = stopwords.words('indonesian')
stopword.extend(["c", "rp", "di", "ok", "tak", "r", "x", "i", "ini", "jadi", "ada", "ke", "dan", "akan", 
"sang", "yang", "ada", "bagi", "untuk", "pada", "dengan", "saat", "tidak", "nak", "tapi", "buat", "semua",
"dari", "ingin", "gara", "saat", "anak", "hey", "bro"])

def clean_text(text, stem=False):
    tokens = []
    for token in text.split():
        if token not in stopword:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return tokens

train['judul_baru'] = train['judul'].apply(lambda x: clean_text(x))
train.head(10)

,judul,link,kategori,label,judul_baru
0,jelang tutup tahun diler mobil sebar diskon ha...,http://www.tribunnews.com/otomotif/2015/12/09/...,Valid,0,"[jelang, tutup, diler, mobil, sebar, diskon, h..."
1,tabir harga suv maserati dibuka tahun depan,http://www.tribunnews.com/otomotif/2015/12/09/...,Valid,0,"[tabir, harga, suv, maserati, dibuka]"
2,tampang skutik honda air blade dirombak total,http://www.tribunnews.com/otomotif/2015/12/09/...,Valid,0,"[tampang, skutik, honda, air, blade, dirombak,..."
3,sinarmas hana finance incar mobil bekas ekspat...,http://www.tribunnews.com/otomotif/2015/12/09/...,Valid,0,"[sinarmas, hana, finance, incar, mobil, bekas,..."
4,orang kaya di indonesia terintimidasi harga ha...,http://www.tribunnews.com/otomotif/2015/12/09/...,Valid,0,"[orang, kaya, indonesia, terintimidasi, harga,..."
5,mazda punya bengkel bodi dan cat di tangerang,http://www.tribunnews.com/otomotif/2015/12/09/...,Valid,0,"[mazda, bengkel, bodi, cat, tangerang]"
6,menunggangi godzilla di sirkuit sodegaura,http://www.tribunnews.com/otomotif/2015/12/09/...,Valid,0,"[menunggangi, godzilla, sirkuit, sodegaura]"
7,mimpi nissan ciptakan mobil yang bisa ngobrol,http://www.tribunnews.com/otomotif/2015/12/09/...,Valid,0,"[mimpi, nissan, ciptakan, mobil, ngobrol]"
8,pandji pragiwaksono jajal bisnis online,http://www.tribunnews.com/bisnis/2015/12/09/pa...,Valid,0,"[pandji, pragiwaksono, jajal, bisnis, online]"
9,rabu sore nilai tukar ditutup melemah jadi rp ...,http://www.tribunnews.com/bisnis/2015/12/09/ra...,Valid,0,"[rabu, sore, nilai, tukar, ditutup, melemah, d..."


Data validation

In [ ]:
#remove punctuation
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
def remove_punctuation(text):
    no_punct = "".join([i for i in text if i not in punc])
    return no_punct

validation['judul'] = validation['judul'].apply(lambda x: remove_punctuation(x))

#lowering text
validation['judul'] = validation['judul'].apply(lambda x: x.lower())

#remove number
def remove_number(text):
    no_number = re.sub(r'\d', '', text)
    return no_number

validation['judul'] = validation['judul'].apply(lambda x: remove_number(x))

#remove emoji
validation["judul"] = validation["judul"].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))

#define stemming
factory = StemmerFactory()
stemmer = factory.create_stemmer()

#define stopwords
stopword = stopwords.words('indonesian')
stopword.extend(["c", "rp", "di", "ok", "tak", "r", "x", "i", "ini", "jadi", "ada", "ke", "dan", "akan", 
"sang", "yang", "ada", "bagi", "untuk", "pada", "dengan", "saat", "tidak", "nak", "tapi", "buat", "semua",
"dari", "ingin", "gara", "saat", "anak", "hey", "bro"])

def clean_text(text, stem=False):
    tokens = []
    for token in text.split():
        if token not in stopword:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return tokens

validation['judul_baru'] = validation['judul'].apply(lambda x: clean_text(x))

validation.head(10)

,judul,link,kategori,label,judul_baru
0,terima rekomendasi bawaslu kpu buka kembali pe...,http://www.tribunnews.com/nasional/2015/08/06/...,Valid,0,"[terima, rekomendasi, bawaslu, kpu, buka, pend..."
1,jokowi lantik sembilan komisioner komisi kejak...,http://www.tribunnews.com/nasional/2015/08/06/...,Valid,0,"[jokowi, lantik, sembilan, komisioner, komisi,..."
2,surat pengunduran diri gus mus ditulistangan h...,http://www.tribunnews.com/nasional/2015/08/06/...,Valid,0,"[surat, pengunduran, gus, mus, ditulistangan, ..."
3,bnpb abu vulkanik gunung raung ke selatantengg...,http://www.tribunnews.com/nasional/2015/08/06/...,Valid,0,"[bnpb, abu, vulkanik, gunung, raung, selatante..."
4,sempat gagal pan berencana ajukan calon lagi l...,http://www.tribunnews.com/nasional/2015/08/06/...,Valid,0,"[gagal, pan, berencana, ajukan, calon, lawan, ..."
5,bnn musnahkan puluhan ribu gram sabu dan ratus...,http://www.tribunnews.com/nasional/2015/08/06/...,Valid,0,"[bnn, musnahkan, puluhan, ribu, gram, sabu, ra..."
6,kabareskrim kaji adanya laporan soal penculika...,http://www.tribunnews.com/nasional/2015/08/06/...,Valid,0,"[kabareskrim, kaji, laporan, penculikan, oc, k..."
7,kpk dipolisikan terkait penculikan oc kaligis ...,http://www.tribunnews.com/nasional/2015/08/06/...,Valid,0,"[kpk, dipolisikan, terkait, penculikan, oc, ka..."
8,saat rekonstruksi pelaku pembunuhan dan pembak...,http://www.tribunnews.com/nasional/2015/08/06/...,Valid,0,"[rekonstruksi, pelaku, pembunuhan, pembakaran,..."
9,setelah pertemuan dengan mui bpjs kesehatan ti...,http://www.tribunnews.com/nasional/2015/08/06/...,Valid,0,"[pertemuan, mui, bpjs, kesehatan, haram]"


In [ ]:
# data testing
#remove punctuation
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
def remove_punctuation(text):
    no_punct = "".join([i for i in text if i not in punc])
    return no_punct

test['judul'] = test['judul'].apply(lambda x: remove_punctuation(x))

#lowering text
test['judul'] = test['judul'].apply(lambda x: x.lower())

#remove number
def remove_number(text):
    no_number = re.sub(r'\d', '', text)
    return no_number

test['judul'] = test['judul'].apply(lambda x: remove_number(x))

#remove emoji
test["judul"] = test["judul"].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))

#define stemming
factory = StemmerFactory()
stemmer = factory.create_stemmer()

#define stopwords
stopword = stopwords.words('indonesian')
stopword.extend(["c", "rp", "di", "ok", "tak", "r", "x", "i", "ini", "jadi", "ada", "ke", "dan", "akan", 
"sang", "yang", "ada", "bagi", "untuk", "pada", "dengan", "saat", "tidak", "nak", "tapi", "buat", "semua",
"dari", "ingin", "gara", "saat", "anak", "hey", "bro"])

def clean_text(text, stem=False):
    tokens = []
    for token in text.split():
        if token not in stopword:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return tokens

test['judul_baru'] = test['judul'].apply(lambda x: clean_text(x))

test.head(10)

,ID,label,tanggal,judul,narasi,nama file gambar,judul_baru
0,71,1,17-Aug-20,pemakaian masker menyebabkan penyakit legionna...,A caller to a radio talk show recently shared ...,71.jpg,"[pemakaian, masker, menyebabkan, penyakit, leg..."
1,461,1,17-Jul-20,instruksi gubernur jateng tentang penilangan ...,Yth.Seluruh Anggota Grup Sesuai Instruksi Gube...,461.png,"[instruksi, gubernur, jateng, penilangan, yg, ..."
2,495,1,13-Jul-20,foto jim rohn jokowi adalah presiden terbaik d...,Jokowi adalah presiden terbaik dlm sejarah ban...,495.png,"[foto, jim, rohn, jokowi, presiden, terbaik, d..."
3,550,1,8-Jul-20,ini bukan politik tapi kenyataan pak jokowi be...,"Maaf Mas2 dan Mbak2, ini bukan politik, tapi k...",550.png,"[politik, kenyataan, jokowi, berhasil, memulan..."
4,681,1,24-Jun-20,foto kadrun kalo lihat foto ini panas dingin,Kadrun kalo lihat foto ini panas dingin . .,681.jpg,"[foto, kadrun, kalo, lihat, foto, panas, dingin]"
5,736,1,17-Jun-20,event promo smartphone jne spesial di bulan juni,"selamat siang teman teman fb ku semuanya🤩,cuma...",736.png,"[event, promo, smartphone, jne, spesial, juni]"
6,1161,1,5-May-20,mereka sudah mempersiapkan diri dengan baik un...,LIHATLAH MEREKA SUDAH MEMPERSIAPKAN DIRI DENGA...,1161.png,"[menguasai, negri]"
7,1597,1,24-Mar-20,foto presiden italia menangis karena tak cukup...,Italia punya fasilitas perawatan kesehatan ter...,1597.png,"[foto, presiden, italia, menangis, lahan, meng..."
8,2098,1,9-Jan-20,kapolres timor tengah utara nusa tenggara tim...,Polisi goblok. Bukanya cpet” d tolongin malah ...,2098.jpg,"[kapolres, timor, utara, nusa, tenggara, timur..."
9,2226,1,24-Dec-19,video polisi china telah menganiaya wanita uig...,Polisi china telah menganiaya wanita uighur le...,2226.png,"[video, polisi, china, menganiaya, wanita, uig..."


# Build and Training Model (LSTM)

In [ ]:
len(train)
len(validation)
len(test)

4231

In [ ]:
train_sentences = train["judul_baru"].tolist()
validation_sentences = validation["judul_baru"].tolist()
test_sentences = test["judul_baru"].tolist()

In [3]:
vocab_size = 2000
embedding_dim = 16
max_length = 100
trunc_type = "post"
padding_type = "post"
oov_tok = "<OOV>"

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

In [ ]:
from keras_preprocessing.sequence import pad_sequences
train_sequences = tokenizer.texts_to_sequences(train_sentences)
training_padded = pad_sequences(train_sequences, maxlen = max_length)

In [ ]:
val_sequences = tokenizer.texts_to_sequences(validation_sentences)
validation_padded = pad_sequences(val_sequences, maxlen = max_length)

In [ ]:
test_sequences = tokenizer.texts_to_sequences(test_sentences)
testing_padded = pad_sequences(test_sequences, maxlen = max_length)

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=128)))
model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

In [ ]:
history = model.fit(training_padded, train["label"], validation_data=(validation_padded, validation["label"]), epochs=5)

In [ ]:
model.save("lstm_model.h5")

In [ ]:
model.save_weights("lstm_weights.h5")

Training history

In [ ]:
# History training

import matplotlib.pyplot as plt

#Summary acc
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()



In [ ]:
# summmary loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

## Model evaluation and prediction

In [ ]:
train_acc = model.evaluate(training_padded, train["label"])
val_acc = model.evaluate(validation_padded, validation["label"])
test_acc = model.evaluate(testing_padded, test["label"])

print("train acc: ", train_acc)
print("val acc: ", val_acc)
print("test acc: ", test_acc)

In [187]:
from tensorflow import keras
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
model = keras.models.load_model("lstm_model.h5")
model.load_weights("lstm_weights.h5")
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, 100, 16)           32000     
                                                                 
 dropout_4 (Dropout)         (None, 100, 16)           0         
                                                                 
 bidirectional_10 (Bidirecti  (None, 256)              148480    
 onal)                                                           
                                                                 
 dense_19 (Dense)            (None, 1)                 257       
                                                                 
Total params: 180,737
Trainable params: 180,737
Non-trainable params: 0
_________________________________________________________________


In [174]:
# preprocess
#remove punctuation
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
def remove_punctuation(text):
    no_punct = "".join([i for i in text if i not in punc])
    return no_punct


#remove number
def remove_number(text):
    no_number = re.sub(r'\d', '', text)
    return no_number


#define stemming

factory = StemmerFactory()
stemmer = factory.create_stemmer()

#define stopwords
stopword = stopwords.words('indonesian')
stopword.extend(["c", "rp", "di", "ok", "tak", "r", "x", "i", "ini", "jadi", "ada", "ke", "dan", "akan", 
"sang", "yang", "ada", "bagi", "untuk", "pada", "dengan", "saat", "tidak", "nak", "tapi", "buat", "semua",
"dari", "ingin", "gara", "saat", "anak", "hey", "bro"])

def clean_text(text, stem=False):
    tokens = []
    for token in text.split():
        if token not in stopword:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return tokens

In [175]:
#predict on new text
def new_predict(text, model):
  text = text.lower()
  text = remove_punctuation(text)
  text = remove_number(text)
  text = text.encode('ascii', 'ignore').decode('ascii')
  text = clean_text(text)
  tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
  tokenizer.fit_on_texts(text)
  word_index = tokenizer.word_index
  text_sequences = tokenizer.texts_to_sequences(text)
  padded = pad_sequences(text_sequences, maxlen = max_length)
  prediction = model.predict(padded)
  avg_pred = prediction.mean()
  if avg_pred >= 0.77:
    text_label = 'Hoax'
  else:
   text_label = 'Valid'
  return avg_pred, text_label


In [181]:
# prediction example
import csv
import pandas as pd

fake_test = pd.read_csv("Data_uji.csv")
#text = fake_test['judul'][120]
text = "Jasad Eril, Anak Ridwan Kamil Gubernur Jawa Barat ditemukan tewas di sungai Aare"
print(text)
avg_pred, text_label= new_predict(text, model)
print()
print('-------Prediction result--------')
print('This news is', text_label.upper())


Jasad Eril, Anak Ridwan Kamil Gubernur Jawa Barat ditemukan tewas di sungai Aare

-------Prediction result--------
This news is VALID
